<a href="https://colab.research.google.com/github/Danya05/optimization_algo/blob/main/torch_resnet_with_diana.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(
            in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


In [3]:
class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, in_planes, planes, stride=1):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,
                               stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, self.expansion *
                               planes, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(self.expansion*planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out

In [4]:
class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512*block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out


In [5]:
def ResNet18():
    return ResNet(BasicBlock, [2, 2, 2, 2])


def ResNet34():
    return ResNet(BasicBlock, [3, 4, 6, 3])


def ResNet50():
    return ResNet(Bottleneck, [3, 4, 6, 3])


def ResNet101():
    return ResNet(Bottleneck, [3, 4, 23, 3])


def ResNet152():
    return ResNet(Bottleneck, [3, 8, 36, 3])


def test():
    net = ResNet18()
    y = net(torch.randn(1, 3, 32, 32))
    print(y.size())

In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn

import torchvision
import torchvision.transforms as transforms

import os
import argparse




device = 'cuda' if torch.cuda.is_available() else 'cpu'
best_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch

# Data
print('==> Preparing data..')
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

trainset = torchvision.datasets.CIFAR10(
    root='./data', train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(
    trainset, batch_size=128, shuffle=True, num_workers=2)
# trainloader_list = list(trainloader)
# trainloader_list.pop()
# trainloader = torch.utils.data.DataLoader(
#     trainloader_list, batch_size=128, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(
    root='./data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(
    testset, batch_size=100, shuffle=False, num_workers=2)
# testloader_list = list(testloader)
# testloader_list.pop()
# testloader = torch.utils.data.DataLoader(
#     testloader_list, batch_size=128, shuffle=True, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck')

==> Preparing data..
Files already downloaded and verified
Files already downloaded and verified


In [7]:
from torch.optim import Optimizer
import numpy as np


net = ResNet(BasicBlock, [3, 4, 6, 3])
net = net.to(device)
if device == 'cuda':
    net = torch.nn.DataParallel(net)
    cudnn.benchmark = True


def compute_gradient(X, y, a, theta):
    X_batch = X.reshape(128, 32 * 32 * 3)
    y_batch = y.reshape(128, 1)
    return (2 * (X_batch.T) @ (X_batch @ (theta) - y_batch) / X_batch.shape[0] +  2 * a * theta).view(-1)

def mse(X, y, theta):
    return torch.mean((torch.matmul(X, theta) - y) ** 2)

def quantization_of_gradient(gradient, dec=3):
    return torch.round(gradient * 10 ** dec) / 10 ** dec


def worker(X_batch, y_batch, a, theta, grad, h, delta, alpha, i):
    with torch.no_grad():
        grad[i] = compute_gradient(X_batch, y_batch, a, theta)
        delta[i] = grad[i] - h[i]
        delta[i] = quantization_of_gradient(delta[i])
        grad[i] = h[i] + delta[i]
        h[i] = h[i] + alpha * delta[i]



class CustomOptimizer(Optimizer):
    def __init__(self, params, learning_rate=0.001, alpha=0.2, num_iterations=1000):
        defaults = dict(lr=learning_rate, alpha=alpha, num_iterations=num_iterations)
        super(CustomOptimizer, self).__init__(params, defaults)


    def step(self, X_batch, y_batch, a, batch_size):
        lr = self.defaults['lr']
        alpha = self.defaults['alpha']
        num_iterations = self.defaults['num_iterations']

        with torch.no_grad():

            theta = torch.zeros(32 * 32 * 3, requires_grad=False).reshape(32 * 32 * 3, 1).to('cuda')
            deltas_parts_from_workers = torch.zeros((2, 32 * 32 * 3), requires_grad=False).to('cuda')
            grad = torch.zeros((2, 32 * 32 * 3), requires_grad=False).to('cuda')
            h = torch.zeros((2, 32 * 32 * 3), requires_grad=False).to('cuda')
            h1 = h.mean(dim=0).to('cuda')

            data1 = []
            data2 = []

            for _ in range(num_iterations):
                # Вызов функции worker для каждого рабочего
                worker(X_batch.reshape(128, 32 * 32 * 3), y_batch.reshape(128, 1), a, theta, grad, h, deltas_parts_from_workers, alpha, 1)

                delta = deltas_parts_from_workers.mean(dim=0).to('cuda')
                gradient = h1 + delta

                data2.append(gradient.norm())
                data1.append(torch.norm(compute_gradient(X_batch, y_batch, a, theta) - gradient))
                theta = theta - lr * gradient.reshape(3 * 32 * 32, 1)
                h1 = h1 + alpha * delta

            # # Обновление параметров в модели
            # for param, theta_val in zip(self.param_groups[0]['params'], theta):
            #     param.data = theta_val
            #     print(param.data)
            #     break

            # weights = net.module.weight

            # # Установите новые значения для весов
            # weights.data = theta.reshape(3, 32, 32)

            # print(len(list(*net.parameters())))
            # print(len(list(net.parameters())[0]))
        return theta

optimizer = CustomOptimizer(net.parameters(), learning_rate=0.001, alpha=0.2, num_iterations=1000)


In [8]:
# net = ResNet(BasicBlock, [3, 4, 6, 3])
# net = net.to(device)
# if device == 'cuda':
#     net = torch.nn.DataParallel(net)
#     cudnn.benchmark = True

In [9]:
#net = ResNet(BasicBlock, [3, 4, 6, 3])
#net = net.to(device)
#if device == 'cuda':
#    net = torch.nn.DataParallel(net)
#    cudnn.benchmark = True


criterion = nn.CrossEntropyLoss()
# optimizer = optim.SGD(net.parameters(), lr=0.01,
#                       momentum=0.9, weight_decay=5e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)


# Training
def train(epoch):
    print('\nEpoch: %d' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        if batch_idx == len(trainloader) - 1:
            continue
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step(inputs, targets, 100, 128)
        #optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()



In [10]:
def test(epoch):
    global best_acc
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            if batch_idx == len(testloader) - 1:
                continue
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

    # Save checkpoint.
    acc = 100.*correct/total
    print(acc)
    if acc > best_acc:
        print('Saving..')
        state = {
            'net': net.state_dict(),
            'acc': acc,
            'epoch': epoch,
        }
        if not os.path.isdir('checkpoint'):
            os.mkdir('checkpoint')
        torch.save(state, './checkpoint/ckpt.pth')
        best_acc = acc


for epoch in range(start_epoch, start_epoch+50):
    train(epoch)
    test(epoch)
    scheduler.step()


Epoch: 0
10.11111111111111
Saving..

Epoch: 1
9.919191919191919

Epoch: 2


KeyboardInterrupt: ignored

In [ ]:
best_acc